In [10]:
from pandas import read_csv
#Load data set
look_back = 2
series_influ_A_df = read_csv('../temp_data/influA_vietnam_last_10_days.csv', index_col=0, engine='python')
train_df = series_influ_A_df.iloc[0:780]
test_df = series_influ_A_df.iloc[780-look_back:]
# series_influ_A_df = series_influ_A_df["Influenza A - All types of surveillance"]

In [11]:
train_df.corr()

,Influenza A - All types of surveillance,temp,humidity,dew,precipcover,tempmax,tempmin,windspeed
Influenza A - All types of surveillance,1.000000,0.199207,0.010973,0.184344,0.005158,0.193676,0.197355,0.104644
temp,0.199207,1.000000,0.154861,0.950260,0.388126,0.992724,0.989928,0.272166
humidity,0.010973,0.154861,1.000000,0.453441,0.426201,0.086811,0.248852,0.118000
dew,0.184344,0.950260,0.453441,1.000000,0.485975,0.921619,0.971720,0.287466
precipcover,0.005158,0.388126,0.426201,0.485975,1.000000,0.354045,0.430823,0.139984
tempmax,0.193676,0.992724,0.086811,0.921619,0.354045,1.000000,0.967902,0.264565
tempmin,0.197355,0.989928,0.248852,0.971720,0.430823,0.967902,1.000000,0.283023
windspeed,0.104644,0.272166,0.118000,0.287466,0.139984,0.264565,0.283023,1.000000


In [12]:
series_influ_A_df.isna().sum()

Influenza A - All types of surveillance    1
temp                                       0
humidity                                   0
dew                                        0
precipcover                                0
tempmax                                    0
tempmin                                    0
windspeed                                  0
dtype: int64

In [13]:
series_influ_A_df = series_influ_A_df[["Influenza A - All types of surveillance", "temp", "tempmax", "dew","windspeed"]]

In [14]:
series_influ_A_df = series_influ_A_df.dropna()

series_influ_A_df = series_influ_A_df.astype('float32')
series_influ_A_df = series_influ_A_df.values
# normalize the dataset
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(series_influ_A_df)
# Create Training and Test

train = dataset[:780, :]
test = dataset[780-look_back:, :]

import numpy as np
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

In [15]:
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import tensorflow as tf

from keras.losses import mean_squared_error

def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(mean_squared_error(y_true, y_pred))
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(5, input_shape=(2, 5)))
model.add(Dense(5))
model.compile(loss=root_mean_squared_error, optimizer='adam')
model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

c:\Users\nghiemxuan\Desktop\nghiem\GR3\GR3\venv\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
778/778 - 2s - 2ms/step - loss: 0.1899
Epoch 2/100
778/778 - 1s - 964us/step - loss: 0.1121
Epoch 3/100
778/778 - 1s - 902us/step - loss: 0.1050
Epoch 4/100
778/778 - 1s - 882us/step - loss: 0.1013
Epoch 5/100
778/778 - 1s - 864us/step - loss: 0.0995
Epoch 6/100
778/778 - 1s - 851us/step - loss: 0.0981
Epoch 7/100
778/778 - 1s - 867us/step - loss: 0.0971
Epoch 8/100
778/778 - 1s - 874us/step - loss: 0.0962
Epoch 9/100
778/778 - 1s - 909us/step - loss: 0.0950
Epoch 10/100
778/778 - 1s - 930us/step - loss: 0.0949
Epoch 11/100
778/778 - 1s - 874us/step - loss: 0.0943
Epoch 12/100
778/778 - 1s - 865us/step - loss: 0.0936
Epoch 13/100
778/778 - 1s - 877us/step - loss: 0.0937
Epoch 14/100
778/778 - 1s - 877us/step - loss: 0.0925
Epoch 15/100
778/778 - 1s - 871us/step - loss: 0.0919
Epoch 16/100
778/778 - 1s - 885us/step - loss: 0.0921
Epoch 17/100
778/778 - 1s - 958us/step - loss: 0.0917
Epoch 18/100
778/778 - 1s - 1ms/step - loss: 0.0915
Epoch 19/100
778/778 - 1s - 1ms/step - lo

In [17]:
model.save("../model/LSTM_4_4.keras")